In [1]:
import streamlit as st
import pandas as pd
import numpy as np
from ydata_profiling import ProfileReport
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib

In [2]:
data = pd.read_excel('Classeur1.xlsx')

In [3]:
data.head()

,user_id,REGION,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,ZONE1,ZONE2,MRG,REGULARITY,TOP_PACK,FREQ_TOP_PACK,CHURN
0,00000bfd7d50f01092811bc0c8d7b0d6fe7c3596,FATICK,K > 24 month,4250.0,15.0,4251.0,1417.0,17.0,4.0,388.0,46.0,1.0,1.0,2.0,NO,54,On net 200F=Unlimited _call24H,8.0,0
1,00000cb4a5d760de88fecb38e2f71b7bec52e834,NaN,I 18-21 month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,4,NaN,NaN,1
2,00001654a9d9f96303d9969d0a4a851714a4bb57,NaN,K > 24 month,3600.0,2.0,1020.0,340.0,2.0,NaN,90.0,46.0,7.0,NaN,NaN,NO,17,On-net 1000F=10MilF;10d,1.0,0
3,00001dd6fa45f7ba044bd5d84937be464ce78ac2,DAKAR,K > 24 month,13500.0,15.0,13502.0,4501.0,18.0,43804.0,41.0,102.0,2.0,NaN,NaN,NO,62,"Data:1000F=5GB,7d",11.0,0
4,000028d9e13a595abe061f9b58f3d76ab907850f,DAKAR,K > 24 month,1000.0,1.0,985.0,328.0,1.0,NaN,39.0,24.0,NaN,NaN,NaN,NO,11,Mixt 250F=Unlimited_call24H,2.0,0


In [4]:
data.columns

Index(['user_id', 'REGION', 'TENURE', 'MONTANT', 'FREQUENCE_RECH', 'REVENUE',
       'ARPU_SEGMENT', 'FREQUENCE', 'DATA_VOLUME', 'ON_NET', 'ORANGE', 'TIGO',
       'ZONE1', 'ZONE2', 'MRG', 'REGULARITY', 'TOP_PACK', 'FREQ_TOP_PACK',
       'CHURN'],
      dtype='object')

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 19 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   user_id         500000 non-null  object 
 1   REGION          302556 non-null  object 
 2   TENURE          500000 non-null  object 
 3   MONTANT         324314 non-null  float64
 4   FREQUENCE_RECH  324314 non-null  float64
 5   REVENUE         331478 non-null  float64
 6   ARPU_SEGMENT    331478 non-null  float64
 7   FREQUENCE       331478 non-null  float64
 8   DATA_VOLUME     253859 non-null  float64
 9   ON_NET          317257 non-null  float64
 10  ORANGE          292015 non-null  float64
 11  TIGO            200597 non-null  float64
 12  ZONE1           39362 non-null   float64
 13  ZONE2           31821 non-null   float64
 14  MRG             500000 non-null  object 
 15  REGULARITY      500000 non-null  int64  
 16  TOP_PACK        290661 non-null  object 
 17  FREQ_TOP_P

In [6]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
MONTANT,324314.0,5543.005939,7186.904976,10.0,1000.0,3000.0,7350.0,470000.0
FREQUENCE_RECH,324314.0,11.552699,13.295211,1.0,2.0,7.0,16.0,133.0
REVENUE,331478.0,5516.355487,7222.181838,1.0,1000.0,3000.0,7382.0,532177.0
ARPU_SEGMENT,331478.0,1838.791235,2407.389459,0.0,333.0,1000.0,2461.0,177392.0
FREQUENCE,331478.0,13.994096,14.714466,1.0,3.0,9.0,20.0,91.0
DATA_VOLUME,253859.0,3363.074522,12377.008674,0.0,0.0,263.0,2902.0,990644.0
ON_NET,317257.0,278.887791,878.235558,0.0,5.0,27.0,157.0,50809.0
ORANGE,292015.0,95.863558,205.899080,0.0,7.0,29.0,99.0,6555.0
TIGO,200597.0,23.059408,63.088716,0.0,2.0,6.0,20.0,3486.0
ZONE1,39362.0,8.164651,41.359560,0.0,0.0,1.0,3.0,2507.0


In [7]:
#copions le dataset dans df
df=data.copy()

In [8]:
# Vérifier les valeurs manquantes
print(df.isnull().sum())

user_id                0
REGION            197444
TENURE                 0
MONTANT           175686
FREQUENCE_RECH    175686
REVENUE           168522
ARPU_SEGMENT      168522
FREQUENCE         168522
DATA_VOLUME       246141
ON_NET            182743
ORANGE            207985
TIGO              299403
ZONE1             460638
ZONE2             468179
MRG                    0
REGULARITY             0
TOP_PACK          209339
FREQ_TOP_PACK     209339
CHURN                  0
dtype: int64


In [9]:
# Vérifier les doublons
print(df.duplicated().sum())

# Supprimer les doublons
df = df.drop_duplicates()


0


**Gestion des valeurs manquantes:**

#### Impute les valeurs manquantes:
- Remplace les valeurs manquantes dans REGION par 'UNKNOWN'.
- Remplace les valeurs manquantes dans toutes les colonnes numériques par 0, sauf pour MONTANT et REVENUE.
- Encode les variables catégorielles:
- Convertit les catégories de TENURE, TOP_PACK et MRG en valeurs numériques.
- Impute les valeurs manquantes par la médiane dans MONTANT et REVENUE
#### Supprime les colonnes ZONE1 et ZONE2.


In [10]:
# Remplacer les valeurs manquantes dans `REGION` par 'UNKNOWN'.
df['REGION'] = df['REGION'].fillna('UNKNOWN')

# Remplacer les valeurs manquantes dans les colonnes numériques par 0, sauf pour `MONTANT` et `REVENUE`.
df[['FREQUENCE_RECH','ARPU_SEGMENT', 'FREQUENCE', 'DATA_VOLUME', 'ON_NET', 'ORANGE', 'TIGO', 'FREQ_TOP_PACK']] = df[['FREQUENCE_RECH', 'ARPU_SEGMENT', 'FREQUENCE', 'DATA_VOLUME', 'ON_NET', 'ORANGE', 'TIGO', 'FREQ_TOP_PACK']].fillna(0)

# Encoder les colonnes catégorielles.
le = LabelEncoder()
df['TENURE'] = le.fit_transform(df['TENURE'])
df['TOP_PACK'] = le.fit_transform(df['TOP_PACK'])
df['MRG'] = le.fit_transform(df['MRG'])

# Calculer la médiane de `MONTANT` et `REVENUE`.
median_montant = df['MONTANT'].median()
median_revenue = df['REVENUE'].median()

# Remplacer les valeurs 0 dans `MONTANT` et `REVENUE` par leurs médianes respectives.
df['MONTANT'] = df['MONTANT'].fillna(median_montant)
df['REVENUE'] = df['REVENUE'].fillna(median_revenue)

# Supprimer les colonnes `ZONE1` et `ZONE2`
df.drop(columns=['ZONE1', 'ZONE2'], inplace=True)

# Afficher les cinq premières lignes du DataFrame après l'imputation
print(df.head())

                                    user_id   REGION  TENURE  MONTANT  \
0  00000bfd7d50f01092811bc0c8d7b0d6fe7c3596   FATICK       7   4250.0   
1  00000cb4a5d760de88fecb38e2f71b7bec52e834  UNKNOWN       5   3000.0   
2  00001654a9d9f96303d9969d0a4a851714a4bb57  UNKNOWN       7   3600.0   
3  00001dd6fa45f7ba044bd5d84937be464ce78ac2    DAKAR       7  13500.0   
4  000028d9e13a595abe061f9b58f3d76ab907850f    DAKAR       7   1000.0   

   FREQUENCE_RECH  REVENUE  ARPU_SEGMENT  FREQUENCE  DATA_VOLUME  ON_NET  \
0            15.0   4251.0        1417.0       17.0          4.0   388.0   
1             0.0   3000.0           0.0        0.0          0.0     0.0   
2             2.0   1020.0         340.0        2.0          0.0    90.0   
3            15.0  13502.0        4501.0       18.0      43804.0    41.0   
4             1.0    985.0         328.0        1.0          0.0    39.0   

   ORANGE  TIGO  MRG  REGULARITY  TOP_PACK  FREQ_TOP_PACK  CHURN  
0    46.0   1.0    0          54     

In [11]:
# Identifier les colonnes catégorielles
colonnes_categorielles = df.select_dtypes(include=['object']).columns

# Appliquer LabelEncoder à chaque colonne catégorielle
label_encoders = {}
for col in colonnes_categorielles:
    le = LabelEncoder()
    df[col] = df[col].astype(str)  # Convertir les valeurs NaN en string pour éviter les erreurs
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le  # Enregistrer le label encoder pour chaque colonne


In [12]:
# choix du features  et de la variable cible 
X = df.drop('CHURN', axis=1)
y = df['CHURN']

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Créer un modèle RandomForestClassifier
model = RandomForestClassifier(random_state=42)

# Entraînement le modèle
model.fit(X_train, y_train)

# prédictions sur l'ensemble de test
y_pred = model.predict(X_test)

# Évaluation du modèle
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print(classification_report(y_test, y_pred))

Accuracy: 0.85
              precision    recall  f1-score   support

           0       0.90      0.93      0.91     81336
           1       0.63      0.54      0.58     18664

    accuracy                           0.85    100000
   macro avg       0.76      0.73      0.75    100000
weighted avg       0.85      0.85      0.85    100000



In [13]:
# Charger le modèle entraîné
# Sauvegarder le modèle entraîné
joblib.dump(model, 'model.pkl')
model = joblib.load('model.pkl')

# Titre de l'application
st.title("Prédiction de l'attrition client")

# Créer des champs de saisie pour les features
region = st.selectbox("Région", df['REGION'].unique())
tenure = st.selectbox("Ancienneté", df['TENURE'].unique())
montant = st.number_input("Montant", min_value=0)
frequence_rech = st.number_input("Fréquence de recharge", min_value=0)
revenue = st.number_input("Revenue", min_value=0)
arpu_segment = st.number_input("ARPU Segment", min_value=0)
frequence = st.number_input("Fréquence", min_value=0)
data_volume = st.number_input("Volume de données", min_value=0)
on_net = st.number_input("On-net", min_value=0)
orange = st.number_input("Orange", min_value=0)
tigo = st.number_input("Tigo", min_value=0)
mrg = st.selectbox("MRG", df['MRG'].unique())
regularity = st.number_input("Régularité", min_value=0)
top_pack = st.selectbox("Top Pack", df['TOP_PACK'].unique())
freq_top_pack = st.number_input("Fréquence Top Pack", min_value=0)

# Bouton pour déclencher la prédiction
if st.button("Prédire"):
    # Créer un DataFrame avec les valeurs saisies
    input_data = pd.DataFrame({
        'REGION': [region],
        'TENURE': [tenure],
        'MONTANT': [montant],
        'FREQUENCE_RECH': [frequence_rech],
        'REVENUE': [revenue],
        'ARPU_SEGMENT': [arpu_segment],
        'FREQUENCE': [frequence],
        'DATA_VOLUME': [data_volume],
        'ON_NET': [on_net],
        'ORANGE': [orange],
        'TIGO': [tigo],
        'MRG': [mrg],
        'REGULARITY': [regularity],
        'TOP_PACK': [top_pack],
        'FREQ_TOP_PACK': [freq_top_pack]
    })

    # Faire la prédiction
    prediction = model.predict(input_data)[0]

    # Afficher le résultat
    if prediction == 0:
        st.success("Le client ne va probablement pas churn.")
    else:
        st.warning("Le client va probablement churn.")


2024-07-04 00:20:41.572 
  command:

    streamlit run C:\Users\7MAKSACOD PC\anaconda3\anaconda\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-07-04 00:20:41.587 Session state does not function when running a script without `streamlit run`
